In [140]:
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

In [144]:
page = requests.get("https://apps.bbmpgov.in/covidbedstatus/")
source = page.content
source = re.sub(r'SR. NO.</td>', "SR. NO.</th>", source.decode('utf-8'))

In [145]:
soup = bs(source, 'html.parser')

In [54]:
df

,name,category,gen_total,HDU_total,ICU_total,ICU_vent_total,gen_available,HDU_available,ICU_available,ICU_vent_available


In [148]:
df = pd.DataFrame(columns=["name", "category", "gen_total", "HDU_total", "ICU_total", "ICU_vent_total", "gen_occupied", "HDU_occupied", "ICU_occupied", "ICU_vent_occupied"])

ids = {
    'governmenthospital': "gov hos",
    'government_medical_college': "gov med",
    'private_hospital': "pri hos",
    'private_medical_college': "pri med",
    'covid_care_centers': "covid",
}

data = {}

structure = ['gen', 'HDU', 'ICU', 'ICU Vent', 'total', 'gen', 'HDU', 'ICU', 'ICU Vent', 'total'] # total, occupied, net

for key, value in ids.items():
    div = soup.find("div", {"id": key})
    if value == "covid":
        print(div)
    rows = div.find_all("tr")
    rows = rows[3:]
    
    
        
    for row in rows:
        columns = row.find_all("td")[1:]
        
        if value == "covid":
            columns = columns[:-10]
        else: 
            columns = columns[:-5]
        values = [column.text for column in columns]
        
        if values[0] == "Total":
            continue
        
        values = {
            "name": values[0],
            "gen_total": int(values[1]),
            "HDU_total": int(values[2]),
            "ICU_total": int(values[3]),
            "ICU_vent_total": int(values[4]),
            "gen_occupied": int(values[6]),
            "HDU_occupied": int(values[7]),
            "ICU_occupied": int(values[8]),
            "ICU_vent_occupied": int(values[9]),
            "category": value
        }
        
        
        df = df.append(values, ignore_index=True)
        
df[["gen_total", "HDU_total", "ICU_total", "ICU_vent_total", "gen_occupied", "HDU_occupied", "ICU_occupied", "ICU_vent_occupied"]] = df[["gen_total", "HDU_total", "ICU_total", "ICU_vent_total", "gen_occupied", "HDU_occupied", "ICU_occupied", "ICU_vent_occupied"]].apply(pd.to_numeric)
        

<div id="covid_care_centers" style="width:1000px; display:block; float:left; margin:10px 1%; min-height:250px">
<table cellpadding="0" cellspacing="0" id="excltable">
<col span="17" width="64"/>
<tr>
<td colspan="23" style="text-align:center" width="513"><h2>Covid    Care Centres (CCC)<a id="ccc"></a></h2></td>
</tr>
<tr>
<th width="64">SR. NO.</th>
<th width="64">Dedicated Covid    Healthcare Centers (DCHCs)</th>
<th colspan="5" valign="middle" width="320">Total    Allocated Beds for C+ Patients</th>
<th colspan="5" valign="middle" width="320">Occupied    Beds for C+ Patients</th>
<th colspan="5" valign="middle" width="320">Net    Available Beds for C+ Patients</th>
<th colspan="5" valign="middle" width="320">Due for Discharge</th>
</tr>
<tr style="background-color:#ffcd378a">
<td><strong>#</strong></td>
<td><strong>Type of facility</strong></td>
<td><strong>Gen</strong></td>
<td><strong>HDU</strong></td>
<td><strong>ICU</strong></td>
<td><strong>ICU Ventl</strong></td>
<td><strong>To

In [149]:
df

,name,category,gen_total,HDU_total,ICU_total,ICU_vent_total,gen_occupied,HDU_occupied,ICU_occupied,ICU_vent_occupied
0,Epedimic Disease Hospital,gov hos,40,0,0,0,14,0,0,0
1,ESIMC PGIMSR MODEL HOSPITAL,gov hos,65,0,15,10,54,0,15,10
2,General Hospital Anekal,gov hos,0,38,3,3,0,0,0,0
3,General Hospital Jayanagar,gov hos,90,20,7,8,43,14,7,6
4,K C General Hospital,gov hos,90,10,4,2,59,10,4,2
...,...,...,...,...,...,...,...,...,...,...
137,Horti Boy's Hostel,covid,200,0,0,0,173,0,0,0
138,Horti Girls's Hostel,covid,138,0,0,0,127,0,0,0
139,Unani college,covid,200,0,0,0,85,0,0,0
140,HAL,covid,178,0,0,0,177,0,0,0


In [136]:
a = df[df.category=="covid"]

In [138]:
np.sum(a.gen_occupied + a.HDU_occupied + a.ICU_occupied + a.ICU_vent_occupied)/np.sum(a.gen_total + a.HDU_total + a.ICU_total + a.ICU_vent_total)












0.7825070159027128

In [90]:
b


0

In [80]:
df.dtypes

name                  object
category              object
gen_total              int64
HDU_total              int64
ICU_total              int64
ICU_vent_total         int64
gen_available          int64
HDU_available          int64
ICU_available          int64
ICU_vent_available     int64
dtype: object